# TRAFFIC LIGHTS
María Rodríguez Palomo

In [15]:
import glob
import os
import sys
import carla
import argparse
import random
import time
import math
import numpy as np
import cv2

#### CREATE WORLD

In [2]:
client = carla.Client('localhost', 2000)
client.set_timeout(2000)
#client.load_world("Town05")
world = client.get_world()
map = world.get_map()

#### CHANGE VIEW

In [181]:
loc2=carla.Transform(carla.Location(x=-124.557922, y=-190.525208, z=10.138501), carla.Rotation(pitch=0.533207, yaw=-1.194565, roll=0.006472))
spectator = world.get_spectator()
spectator_pos = carla.Transform(loc2.location + carla.Location(x=-10,y=0,z=4),carla.Rotation(yaw=loc2.rotation.yaw))
spectator.set_transform(spectator_pos)

#### VEHICLE CREATION

In [201]:
blueprint_library = world.get_blueprint_library()
bp = random.choice(blueprint_library.filter('vehicle.tesla.model3'))

if bp.has_attribute('color'):
    color = bp.get_attribute('color').recommended_values[2]
    bp.set_attribute('color', '255,0,255')

vehicle = world.spawn_actor(bp, loc2)

#### CONSTANT SPEED

In [183]:
def maintain_speed(s,preferred_speed,speed_threshold):
    if s>=preferred_speed:
        return 0
    elif s < preferred_speed - speed_threshold:
        return 0.8
    else:
        return 0.4

#### ANGLE

In [184]:
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

def get_angle(car,wp):
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))
    return degrees

In [185]:
def steer(predicted_angle,MAX_STEER_DEGREES):
    if predicted_angle<-180:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 180:
        predicted_angle = predicted_angle-360
    steer_input = predicted_angle
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    steer_input = steer_input/75
    return steer_input

#### TRAFFIC LIGHT

In [199]:
def is_traffic_light_red(vehicle, world):
    location = vehicle.get_location()
    actor_list = world.get_actors()

    for actor in actor_list:
        if actor.type_id.startswith("traffic.traffic_light"):
            traffic_light_location = actor.get_location()
            distance = location.distance(traffic_light_location)
            traffic_light_state = actor.get_state()
            traffic_light_waypoint = world.get_map().get_waypoint(traffic_light_location) 
            if traffic_light_waypoint.road_id == 2353 and traffic_light_waypoint.lane_id == 1:
                if distance < 37.0:
                    if traffic_light_state == carla.TrafficLightState.Red:
                        return True
                    elif traffic_light_state == carla.TrafficLightState.Green:
                        return False
                    
            if traffic_light_waypoint.road_id == 2043 and traffic_light_waypoint.lane_id == 1:
                print(distance)
                if distance < 70.0:
                    if traffic_light_state == carla.TrafficLightState.Red:
                        return True
                    elif traffic_light_state == carla.TrafficLightState.Green:
                        return False

#### BRAKE

In [191]:
def freno(light,brake_intensity,vehicle,estimated_throttle,steer_input):
    if light:
        control = carla.VehicleControl(brake=brake_intensity)
        vehicle.apply_control(control)
    else:    
        vehicle.apply_control(carla.VehicleControl(throttle= estimated_throttle, steer=steer_input))

#### DRIVING

In [202]:
MAX_STEER_DEGREES = 20
brake_intensity = 0.8 
preferred_speed = 90
speed_threshold = 2

while True:
    loc=vehicle.get_location()
    current_w = map.get_waypoint(loc)
    waypoint_separation = 5
    next_w0 = list(current_w.next(waypoint_separation))[0]
    next_w = map.get_waypoint(next_w0.transform.location,project_to_road=True, lane_type=(carla.LaneType.Driving))
    
    velocity = vehicle.get_velocity()
    speed = round(3.6 *math.sqrt(velocity.x**2+ velocity.y**2+velocity.z**2),0)
    estimated_throttle = maintain_speed(speed,preferred_speed,speed_threshold)
    predicted_angle = get_angle(vehicle,next_w)
    steer_input=steer(predicted_angle,MAX_STEER_DEGREES)
    if abs(steer_input) > 0.05:
        estimated_throttle=0.2
    light=is_traffic_light_red(vehicle, world)
    freno(light,brake_intensity,vehicle,estimated_throttle,steer_input)

171.6267547607422
171.6267547607422
171.6267547607422
171.6267547607422
171.6267547607422
171.626708984375
171.626708984375
171.626708984375
171.626708984375
171.62667846679688
171.62667846679688
171.62667846679688
171.62667846679688
171.62667846679688
171.62667846679688
171.6266632080078
171.6266632080078
171.6266632080078
171.6266632080078
171.6266632080078
171.6266632080078
171.6266632080078
171.6266632080078
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.62664794921875
171.6266479492

44.55818557739258
44.55818557739258
44.55818557739258
43.62977600097656
43.62977600097656
43.62977600097656
42.69385528564453
42.69385528564453
42.69385528564453
42.69385528564453
41.829647064208984
41.829647064208984
41.829647064208984
41.829647064208984
41.02384567260742
41.02384567260742
41.02384567260742
40.241241455078125
40.241241455078125
40.241241455078125
40.241241455078125
39.480491638183594
39.480491638183594
39.480491638183594
39.480491638183594
38.786136627197266
38.786136627197266
38.786136627197266
38.786136627197266
38.13557052612305
38.13557052612305
38.13557052612305
37.471275329589844
37.471275329589844
37.471275329589844
37.471275329589844
36.861305236816406
36.861305236816406
36.861305236816406
36.283912658691406
36.283912658691406
36.283912658691406
35.7225227355957
35.7225227355957
35.7225227355957
35.7225227355957
35.17378234863281
35.17378234863281
35.17378234863281
35.17378234863281
34.68305969238281
34.68305969238281
34.68305969238281
34.23823547363281
34.238

32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311035156
32.849281311

KeyboardInterrupt: 

#### DESTROY

In [203]:
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()